In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
#allows to print the dataframe nicely
from IPython.core import display as ICD

In [3]:
# import additional packages to insepct data
import pandas as pd
import os 
import random 
from zipfile import ZipFile

In [4]:
# import helping functions
from proj1_helpers import load_csv_data

In [5]:
# import files
data_folder='./data/'
zip_file = ZipFile(data_folder+'all.zip')
# zip file creates a list of files with certain properties
zip_file.infolist()

[<ZipInfo filename='train.csv' compress_type=deflate file_size=48103379 compress_size=17670847>,
 <ZipInfo filename='test.csv' compress_type=deflate file_size=109335584 compress_size=40089137>,
 <ZipInfo filename='sample-submission.csv' compress_type=deflate file_size=5682394 compress_size=1263762>]

In [6]:
# now we want to access the 'filename' property in the zipfile variable
# and we create a dictionary of dataframe
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}
df_train=dfs['train.csv']
df_test=dfs['test.csv']
df_sample_submission=dfs['sample-submission.csv']

In [7]:
ICD.display(df_train['Prediction'].head())
ICD.display(df_train.head())

0    s
1    b
2    b
3    b
4    b
Name: Prediction, dtype: object

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,...,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226
2,100002,b,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,...,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251
3,100003,b,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,...,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000
4,100004,b,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,...,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000


In [8]:
yb, input_data, ids=np.array(df_train['Prediction']), np.array(df_train.iloc[:,2:]), np.array(df_train['Id'])

In [9]:
ICD.display(input_data)
ICD.display(df_train.iloc[:,2:].head())

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,197.760,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,125.157,...,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226
2,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,197.814,...,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251
3,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,75.968,...,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000
4,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,57.983,...,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000


In [10]:
# transform yb into numerical values
yb[np.where(yb=='b')] = -1
yb[np.where(yb=='s')] = 1
input_data.shape[1]

30

Build Polynomial

In [28]:
# from helpers import polynomial_regression
from helpers import build_poly
degree=10
tx=build_poly(input_data[:50,:], degree)
ICD.display(tx)
ICD.display(tx.shape)
np.transpose(tx,(2,1,0)).shape

array([[[ 1.00000000e+00,  1.38470000e+02,  1.91739409e+04, ...,
          1.35159177e+17,  1.87154912e+19,  2.59153407e+21],
        [ 1.00000000e+00,  1.60937000e+02,  2.59007180e+04, ...,
          4.50035954e+17,  7.24274363e+19,  1.16562543e+22],
        [ 1.00000000e+00, -9.99000000e+02,  9.98001000e+05, ...,
          9.92027944e+23, -9.91035916e+26,  9.90044880e+29],
        ...,
        [ 1.00000000e+00,  1.30079000e+02,  1.69205462e+04, ...,
          8.19704872e+16,  1.06626390e+19,  1.38698542e+21],
        [ 1.00000000e+00,  1.67534000e+02,  2.80676412e+04, ...,
          6.20616992e+17,  1.03974447e+20,  1.74192550e+22],
        [ 1.00000000e+00,  9.54070000e+01,  9.10249565e+03, ...,
          6.86502179e+15,  6.54971134e+17,  6.24888310e+19]],

       [[ 1.00000000e+00,  5.16550000e+01,  2.66823903e+03, ...,
          5.06872731e+13,  2.61825109e+15,  1.35245760e+17],
        [ 1.00000000e+00,  6.87680000e+01,  4.72903782e+03, ...,
          5.00139494e+14,  3.43935927e

(30, 50, 11)

(11, 50, 30)

In [66]:
from helpers import ridge_regression
lambda_=10
aI = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
# np.tensordot(np.transpose(tx,(0,2,1)),tx, axes=([0],[0]))
tx_t=np.transpose(tx,(0,2,1))
ICD.display(tx_t.shape)
ICD.display(tx.shape)

a = np.tensordot(tx_t, tx, axes=(1,2))

a = np.transpose(tx,(0,2,1)).dot(tx) + aI
b = tx.T.dot(y)
return np.linalg.solve(a, b)

(30, 11, 50)

(30, 50, 11)

ValueError: operands could not be broadcast together with shapes (30,11,30,11) (50,50) 

In [94]:
for i in range(5):
    for j in range(2):
        for k in range(3):
            for n in range(4):
                d[i,j] += a[k,n,i] * b[n,k,j]
c == d

(250000, 301)